In [1]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("pyspark")
sc = SparkContext(conf=conf)

In [2]:
btc_raw = sc.textFile("graph_datasets/soc-sign-bitcoinalpha.csv")
btc_raw.take(10)
G = btc_raw.map(lambda x: x.split(",")).map(lambda x: (int(x[0]), int(x[1]))).flatMap(lambda x: [x, (x[1], x[0])]).groupByKey().mapValues(lambda x: set(x))

# Algorithm

Things to do
1. SQL version
2. Maybe improve seed_propragation

# Experiement

Variables: Size of the file, Numbers of the clusters

Output metrices: average of CPU times and average of Wll times


## 1. Self Comparison

2 versions of Min_Selection_Step & Pruning_Step. [Input Graph, Output raw Tree] (parent, child)

3 versions of findSeeds step [Input raw Tree, Output Seeds] (Seeds)

1 version of Seed_Propration step [Input raw Tree and Seeds, Output complete Tree] : (vertex, {seed, child})

## 2. Competition 

Google's version

Pyspark's version

Paper's version

Our pyspark version

Our SQL version

# Summary


1.Min_Selection_Step & Pruning_Step

| Architecture  |CPU times| Wall times |
|:------------- |--------:|:----------:|
| Prototyoe     | 428 ms  | 7.79 s     | 
| Brocasting    | 224 ms  | 1.29 s     | 

2.findSeeds step

| Architecture  |CPU times| Wall times |
|:------------- |--------:|:----------:|
| findSeeds1    | 293 ms  | 2.15 s     | 
| findSeeds2    | 378 ms  | 1.82 s     | 
| findSeeds3    | 383 ms  | 1.89 s     | 

3.Seed_Propration step

| Architecture  |CPU times| Wall times |
|:------------- |--------:|:----------:|
| Prototyoe     | 157 ms  | 3.29 s     | 


## Broadcasting vs Prototype: the slowest version in the world

In [25]:
#Version with broadcasting in the addEdge step on both Min_Selection_Step and Pruning_Step

def Min_Selection_Step(G): #dictionary format RDD
    v_min = G.map(lambda x: (x[0], min(x[1] | {x[0]})))
    NN_G_u = G.map(lambda x: (x[0], (x[1] | {x[0]})))
    
    #Broadcasting
    v_min_bc = sc.broadcast(dict(v_min.collect()))
    addEdge = NN_G_u.map(lambda x: (x[0], (x[1], v_min_bc.value[x[0]]))).flatMap(lambda x: [(y, x[1][1]) for y in x[1][0]])
    #Without broadcasting
    #addEdge = NN_G_u.join(v_min).flatMap(lambda x: [(y, x[1][1]) for y in x[1][0]])
    H = addEdge.groupByKey().map(lambda x: (x[0], set(x[1])))#.filter(lambda x: len(x[1]) > 1)
    return H

def Pruning_Step(H):
    H_filtered = H.filter(lambda x: len(x[1]) > 1)
    v_min = H_filtered.map(lambda x: (x[0], min(x[1])))
    NN_H_u = H_filtered.map(lambda x: (x[0], x[1] - {min(x[1])} ))
    
    #Broadcasting
    v_min_bc = sc.broadcast(dict(v_min.collect()))
    addEdge2 = NN_H_u.map(lambda x: (x[0], (x[1], v_min_bc.value[x[0]]))).flatMap(lambda x: [(x[1][1], y) for y in x[1][0]])
    #Without broadcasting
    #addEdge2 = NN_H_u.join(v_min).flatMap(lambda x: [(x[1][1], y) for y in x[1][0]])
    G = addEdge2.flatMap(lambda x: [x, (x[1], x[0])]).groupByKey().map(lambda x: (x[0], set(x[1])))
    return G 

def cracker(G_i):
    count = 0
    while G_i.take(1):
        count += 1
        H_i = Min_Selection_Step(G_i)
        G_i = Pruning_Step(H_i)
        
    return count


In [26]:
#Version without broadcasting in the addEdge step on both Min_Selection_Step and Pruning_Step

def Min_Selection_Step2(G): #dictionary format RDD
    v_min = G.map(lambda x: (x[0], min(x[1] | {x[0]})))
    NN_G_u = G.map(lambda x: (x[0], (x[1] | {x[0]})))
    addEdge = v_min.join(NN_G_u)
    H = addEdge.flatMap(lambda x: [(y, x[1][0]) for y in x[1][1]]).groupByKey().map(lambda x: (x[0], set(x[1])))
    return H

def Pruning_Step2(H):
    H_filtered = H.filter(lambda x: len(x[1]) > 1)
    v_min = H_filtered.map(lambda x: (x[0], min(x[1])))
    NN_H_u = H_filtered.map(lambda x: (x[0], x[1] - {min(x[1])} ))
    addEdge2 = v_min.join(NN_H_u).flatMap(lambda x: [(x[1][0], y) for y in x[1][1]])
    G = addEdge2.flatMap(lambda x: [x, (x[1], x[0])]).groupByKey().map(lambda x: (x[0], set(x[1])))
    return G 

def cracker2(G_i):
    count = 0
    while G_i.take(1):
        count += 1
        H_i = Min_Selection_Step2(G_i)
        G_i = Pruning_Step2(H_i)
        
    return count


In [27]:
%%time
cracker(G)

CPU times: user 189 ms, sys: 35.3 ms, total: 224 ms
Wall time: 1.29 s


3

In [28]:
%%time
cracker2(G)

CPU times: user 365 ms, sys: 62.7 ms, total: 428 ms
Wall time: 7.79 s


3

In [3]:
def Min_Selection_Step(G): #dictionary format RDD
    v_min = G.map(lambda x: (x[0], min(x[1] | {x[0]})))
    NN_G_u = G.map(lambda x: (x[0], (x[1] | {x[0]})))
    addEdge1 = v_min.cogroup(NN_G_u).map(lambda x :(x[0], ( list(x[1][0]), list(x[1][1])))) #if it is possible to reduce to one MapReduce job
    H = addEdge1.flatMap(lambda x: [(x[1][0][0], y) for y in x[1][1][0]]).map(lambda x: (x[1], x[0])).groupByKey().map(lambda x: (x[0], set(x[1])))#.filter(lambda x: len(x[1]) > 1)
    return H

def Pruning_Step(H, T):
    H_filtered = H.filter(lambda x: len(x[1]) > 1)
    v_min_filtered = H_filtered.map(lambda x: (x[0], min(x[1])))
    NN_H_u = H_filtered.map(lambda x: (x[0], x[1] - {min(x[1])} ))
    addEdge2 = v_min_filtered.cogroup(NN_H_u).map(lambda x :(x[0], ( list(x[1][0]), list(x[1][1]))))
    G = addEdge2.flatMap(lambda x: [(x[1][0][0], y) for y in x[1][1][0]]).flatMap(lambda x: [x, (x[1], x[0])]).groupByKey().map(lambda x: (x[0], set(x[1])))
    
    
    #deactiviation
    deactiveNodes = H.filter(lambda x: x[0] not in x[1]).map(lambda x: (x[0], None))
    v_min = H.map(lambda x: (x[0], min(x[1])))
    addEdge3 = deactiveNodes.join(v_min).map(lambda x: (x[1][1], x[0]))
    T = T.union(addEdge3)

    
    return [G, T]

def findSeeds(T):
    T_rev = T.map(lambda x:(x[1], x[0]))
    A = T.keys().distinct().map(lambda x:(x,1))
    B = T_rev.keys().distinct().map(lambda x:(x,1))
    return A.leftOuterJoin(B).filter(lambda x: not x[1][1]).map(lambda x:x[0])

def Cracker(G):
    n = 0
    T = sc.parallelize([])
    while G.take(1):
        n += 1
        print(n)
        H = Min_Selection_Step(G)
        G, T = Pruning_Step(H, T)
    
    return [T, findSeeds(T)]

In [4]:
%%time 
#Cracker with findSeeds
T, Seeds = Cracker(G)
Seeds.collect()

1
2
3
CPU times: user 710 ms, sys: 138 ms, total: 848 ms
Wall time: 26.9 s


In [5]:
Seeds.collect()

[1, 1389, 3228, 1870, 5837]

# Finding seeds from tree (3 different findSeed functions)

In [6]:
#btc_raw = sc.parallelize([(0,1), (1,2), (2,5), (5,8), (7,8), (3,7), (3,4), (3,6), (10,11), (10,12), (12,13)])
#G = btc_raw.flatMap(lambda x: [x, (x[1], x[0])]).groupByKey().mapValues(lambda x: set(x))

In [7]:
def Min_Selection_Step(G): #dictionary format RDD
    v_min = G.map(lambda x: (x[0], min(x[1] | {x[0]})))
    NN_G_u = G.map(lambda x: (x[0], (x[1] | {x[0]})))
    
    #Broadcasting
    v_min_bc = sc.broadcast(dict(v_min.collect()))
    addEdge = NN_G_u.map(lambda x: (x[0], (x[1], v_min_bc.value[x[0]]))).flatMap(lambda x: [(y, x[1][1]) for y in x[1][0]])
    #Without broadcasting
    #addEdge = NN_G_u.join(v_min).flatMap(lambda x: [(y, x[1][1]) for y in x[1][0]])
    #H = addEdge.groupByKey().map(lambda x: (x[0], set(x[1])))
    H = addEdge.groupByKey().mapValues(lambda x: set(x))
    return H

def Pruning_Step(H, T):
    
    #minimum node of the neighborhood: shared for following parts
    v_min = H.map(lambda x: (x[0], min(x[1])))
    v_min_bc = sc.broadcast(dict(v_min.collect())) #Broadcasting v_min
    
    #---------------G construction-------------------
    H_filtered = H.filter(lambda x: len(x[1]) > 1)
    #NN_H_u = H_filtered.map(lambda x: (x[0], x[1] - {min(x[1])} ))
    NN_H_u = H_filtered.mapValues(lambda x: x - {min(x)} )
    #With Broadcasting
    addEdge2=NN_H_u.map(lambda x:(x[0],(x[1],v_min_bc.value[x[0]]))).flatMap(lambda x:[(x[1][1],y) for y in x[1][0]])
    #Without broadcasting
    #addEdge2 = NN_H_u.join(v_min).flatMap(lambda x: [(x[1][1], y) for y in x[1][0]])
    G = addEdge2.flatMap(lambda x: [x, (x[1], x[0])]).groupByKey().mapValues(lambda x: set(x))
    
    #---------------Tree construction--------------
    deactiveNodes = H.filter(lambda x: x[0] not in x[1]).mapValues(lambda x: None)
    #Without broadcasting
    #addEdge3 = deactiveNodes.join(v_min).map(lambda x: (x[1][1], x[0]))
    #With Broadcasting
    addEdge3 = deactiveNodes.map(lambda x: (x[0], (x[1], v_min_bc.value[x[0]]))).map(lambda x: (x[1][1], x[0]))
    T = T.union(addEdge3)

    return [G, T]

#Finding seeds
def findSeeds(T): 
    keys = T
    values = T.map(lambda x:(x[1], x[0]))
    return keys.subtractByKey(values).keys().distinct()

def findSeeds1(T):
    keys = T.keys().distinct().map(lambda x:(x,1))
    values = T.values().distinct().map(lambda x:(x,1))
    return keys.subtractByKey(values).keys()

def findSeeds2(T):
    T_inv = T.map(lambda x:(x[1], x[0]))
    A = T.keys().distinct().map(lambda x:(x,1))        #Each distinct is a reduceByKey
    B = T_inv.keys().distinct().map(lambda x:(x,1))
    return A.leftOuterJoin(B).filter(lambda x: not x[1][1]).keys()


In [8]:
def Cracker(G):
    n = 0
    T = sc.parallelize([])
    while G.take(1):
        n += 1
        H = Min_Selection_Step(G)
        G, T = Pruning_Step(H, T)
    
    return [T, findSeeds2(T)]

In [9]:
%%time 
#Cracker with findSeeds
T, Seeds = Cracker(G)
Seeds.collect()

CPU times: user 254 ms, sys: 38.8 ms, total: 293 ms
Wall time: 2.15 s


In [10]:
Seeds.collect()

[1, 3228, 1389, 1870, 5837]

# Tracking activeness and seeds

In [11]:
#btc_raw = sc.parallelize([(0,1), (1,2), (2,5), (5,8), (7,8), (3,7), (3,4), (3,6), (10,11), (10,12), (12,13)])
#G2 = btc_raw.flatMap(lambda x: [x, (x[1], x[0])]).groupByKey().mapValues(lambda x: (set(x), True))

#for big sets
G2 = btc_raw.map(lambda x: x.split(",")).map(lambda x: (int(x[0]), int(x[1]))).flatMap(lambda x: [x, (x[1], x[0])]).groupByKey().mapValues(lambda x: (set(x), True))

In [12]:
def Min_Selection_Step(G): #dictionary format RDD
    v_min = G.map(lambda x: (x[0], min(x[1][0] | {x[0]})))
    NN_G_u = G.map(lambda x: (x[0], (x[1][0] | {x[0]}, x[1][1])))
    
    #Broadcasting
    v_min_bc = sc.broadcast(dict(v_min.collect()))
    addEdge = NN_G_u.map(lambda x: (x[0], (x[1][0], v_min_bc.value[x[0]], x[1][1])))
    addEdge1 = addEdge.flatMap(lambda x: [(y, (x[1][1], x[1][2])) for y in x[1][0]])
    
    temp = addEdge1.groupByKey().map(lambda x: (x[0], (list(x[1]))))
    H = temp.map(lambda x: (x[0], list(zip(*x[1])))).mapValues(lambda x: (set(x[0]), all(x[1])))
    return H

def Pruning_Step(H, T, Seeds):
    
    #minimum node of the neighborhood: shared for following parts
    v_min = H.map(lambda x: (x[0], min(x[1][0])))
    v_min_bc = sc.broadcast(dict(v_min.collect())) #Broadcasting v_min

    #---------------G construction-------------------
    H_filtered = H.filter(lambda x: len(x[1][0]) > 1)
    NN_H_u = H_filtered.mapValues(lambda x: (x[0] - {min(x[0])}, x[1] ))
    #With Broadcasting
    addEdge2=NN_H_u.map(lambda x:(x[0],(x[1][0],v_min_bc.value[x[0]], x[1][1]))).flatMap(lambda x:[(x[1][1],(y, x[1][2])) for y in x[1][0]])

    temp = addEdge2.flatMap(lambda x: [x, (x[1][0], (x[0],x[1][1]))]).groupByKey().mapValues(lambda x: list(x))
    G = temp.mapValues(lambda x: list(zip(*x))).mapValues(lambda x: (set(x[0]), all(x[1])))
    
    #---------------Tree construction--------------
    #The deactivated Nodes do not appear in G_{t+1}
    deactiveNodes = H.filter(lambda x: x[0] not in x[1][0]).mapValues(lambda x: False)
    #With Broadcasting
    addEdge3 = deactiveNodes.map(lambda x: (x[0], (x[1], v_min_bc.value[x[0]]))).map(lambda x: (x[1][1], x[0]))
    T = T.union(addEdge3)
    
    #--------------Find Seed-----------------
    #Elements in H with neighborhood from G_{t+1}
    NN_G_H = H.cogroup(G).mapValues(lambda x: arr( (list(x[0]), list(x[1])) ) ) 
    
    #---->Not sure if it is necessary to use True/False
    #deactivated = NN_G_H.cogroup(deactiveNodes).map(lambda x: (x[0], arr2( (list(x[1][0]), list(x[1][1])) ) ) )
    #seed = deactivated.filter(lambda x: (len(x[1][0]) <= 1) & (x[0] in x[1][0]) & x[1][1])
    
    seed = NN_G_H.filter(lambda x: (len(x[1][0]) <= 1) & (x[0] in x[1][0]))
    Seeds = Seeds.union(seed)
    return [G, T, Seeds]

def arr(value):
    if not value[1]:
        return value[0][0]
    else:
        temp = list(zip(*[value[0][0], value[1][0]]))
        return temp[0][0].union(temp[0][1]), all(temp[1])

def arr2(value):
    if not value[1]:
        return value[0][0]
    else:
        return (value[0][0][0], False)


In [13]:
def Cracker(G):
    n = 0
    T = sc.parallelize([])
    Seeds = sc.parallelize([])
    while G.take(1):
        n += 1
        H = Min_Selection_Step(G)
        G, T, Seeds = Pruning_Step(H, T, Seeds)
    
    return [T, Seeds.keys()]

In [14]:
%%time
T, Seeds = Cracker(G2)
Seeds.collect()

CPU times: user 335 ms, sys: 43.2 ms, total: 378 ms
Wall time: 1.82 s


In [15]:
Seeds.collect()

[3228, 1389, 5837, 1870, 1]

# Tracking seeds without activeness

In [16]:
#btc_raw = sc.parallelize([(0,1), (1,2), (2,5), (5,8), (7,8), (3,7), (3,4), (3,6), (10,11), (10,12), (12,13)])
G = btc_raw.map(lambda x: x.split(",")).map(lambda x: (int(x[0]), int(x[1]))).flatMap(lambda x: [x, (x[1], x[0])]).groupByKey().mapValues(lambda x: set(x))

In [17]:
def Min_Selection_Step(G): #dictionary format RDD
    v_min = G.map(lambda x: (x[0], min(x[1] | {x[0]})))
    NN_G_u = G.map(lambda x: (x[0], x[1] | {x[0]}))
    #Broadcasting
    v_min_bc = sc.broadcast(dict(v_min.collect()))
    addEdge = NN_G_u.map(lambda x: (x[0], (x[1], v_min_bc.value[x[0]])) )
    addEdge1 = addEdge.flatMap(lambda x: [(y, x[1][1]) for y in x[1][0]])
    #Without broadcasting
    #addEdge = NN_G_u.join(v_min).flatMap(lambda x: [(y, x[1][1]) for y in x[1][0]])

    H = addEdge1.groupByKey().mapValues(lambda x: set(x))
    return H

def Pruning_Step(H, T, Seeds):
    #minimum node of the neighborhood: shared for following parts
    v_min = H.mapValues(lambda x: min(x))
    v_min_bc = sc.broadcast(dict(v_min.collect())) #Broadcasting v_min
    
    #---------------G construction-------------------
    H_filtered = H.filter(lambda x: len(x[1]) > 1)
    NN_H_u = H_filtered.mapValues(lambda x: x - {min(x)} )
    #With Broadcasting
    addEdge2=NN_H_u.map(lambda x:(x[0],(x[1],v_min_bc.value[x[0]]))).flatMap(lambda x:[(x[1][1],y) for y in x[1][0]])
    #Without broadcasting
    #addEdge2 = NN_H_u.join(v_min).flatMap(lambda x: [(x[1][1], y) for y in x[1][0]])
    G = addEdge2.flatMap(lambda x: [x, (x[1], x[0])]).groupByKey().mapValues(lambda x: set(x))
    
    #---------------Tree construction--------------
    #The deactivated Nodes do not appear in G_{t+1}
    deactiveNodes = H.filter(lambda x: x[0] not in x[1]).mapValues(lambda x: False)
    #Without broadcasting
    #addEdge3 = deactiveNodes.join(v_min).map(lambda x: (x[1][1], x[0]))
    #With Broadcasting
    addEdge3 = deactiveNodes.map(lambda x: (x[0], (x[1], v_min_bc.value[x[0]]))).map(lambda x: (x[1][1], x[0]))
    T = T.union(addEdge3)

    #--------------Find Seed-----------------
    #Elements in H with neighborhood from G_{t+1}
    NN_G_H = H.cogroup(G).mapValues(lambda x: (list(x[0]), list(x[1])) ).mapValues(lambda x: set_join(x) )

    #Not sure is necessary to use True/False
    #deactivated = NN_G_H.cogroup(deactiveNodes).map(lambda x: (x[0], (list(x[1][0]), list(x[1][1])) ))
    #seed = deactivated.filter(lambda x: (len(x[1][0]) <= 1) & (x[0] in x[1][0]) & x[1][1]) 
    
    seed = NN_G_H.filter(lambda x: (len(x[1]) <= 1) & (x[0] in x[1]))
    Seeds = Seeds.union(seed)

    return [G, T, Seeds]


def set_join(value):
    if not value[1]:
        return value[0][0]
    else:
        return value[0][0] | value[1][0]


In [18]:
def Cracker(G):
    n = 0
    T = sc.parallelize([])
    Seeds = sc.parallelize([])
    while G.take(1):
        n += 1
        H = Min_Selection_Step(G)
        G, T, Seeds = Pruning_Step(H, T, Seeds)
    
    return [T, Seeds.keys()]

In [19]:
%%time
T, Seeds = Cracker(G)
Seeds.collect()

CPU times: user 336 ms, sys: 47.3 ms, total: 383 ms
Wall time: 1.89 s


In [20]:
Seeds.collect()

[1870, 3228, 1389, 5837, 1]

## Seed_Propragation

In [21]:
def Seed_Propragation(T, seed): 
    seed = seed.map(lambda x: (x, x))  
    T_seed = sc.parallelize([(-1, (None, -1))])                       
    
    while T_seed.map(lambda x: (x[1])).lookup(None):
        T_seed = seed.rightOuterJoin(T)
        seed = T_seed.map(lambda x: (x[1][1], x[1][0])).union(seed)
        
    return T_seed

In [22]:
%%time
T_prop = Seed_Propragation(T, Seeds)
T_prop.collect()

#Wht Wall time takes inproprotional time w.r.t CPU total time compared to others?

CPU times: user 131 ms, sys: 26.2 ms, total: 157 ms
Wall time: 3.29 s


In [23]:
T_prop.take(10)

[(40, (1, 2724)),
 (40, (1, 2728)),
 (40, (1, 1612)),
 (40, (1, 2478)),
 (40, (1, 2641)),
 (40, (1, 7453)),
 (80, (1, 3034)),
 (80, (1, 3239)),
 (360, (1, 2885)),
 (121, (1, 828))]